<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [1]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus

mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('ramayana_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bible_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])


with open ('bible_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('ramayan_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

print(len(mal_lines))
print(len(sanskrit_lines))

C:\Users\Glitch\Sanskrit2mal\parallel-corpus
18728
18728


In [6]:
import random
c = list(zip(sanskrit_lines, mal_lines))
#random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:18728]
train_text_mal = mal_lines[:18728]

dev_text_sa = sanskrit_lines[-728:]
dev_text_mal = mal_lines[-728:]

In [7]:
print(len(train_text_mal))
print(len(dev_text_mal))

print(len(train_text_sa))
print(len(dev_text_sa))

18728
728
18728
728


In [10]:
print(dev_text_mal[0])
print(train_text_mal[0])

യേശു വീണ്ടും കപ്പൽ കയറി മറുകരയിൽ എത്തിയപ്പോൾ വളരെ ആളുകൾ അവന്റെ അടുക്കൽ വന്നുകൂടി; അവൻ കടലിന്നരികെ ആയിരുന്നു.
ധൃതരാഷ്ട്രര്‍ ചോദിച്ചു: സഞ്ജയ, ധ‍ര്‍മ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്‍ ഒരുമിച്ചു ചേര്‍ന്നവരും യുദ്ധം ചെയ്യാന്‍ കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്‍മാരും പാണ്ഡവന്‍മാരും എന്ത് ചെയ്തു?


In [47]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [48]:
# Reading the data and forming pairs
def readLangs(lang1, lang2):
    print("Reading lines...")
    
    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_mal[i]] for i in range(len(train_text_sa))]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [49]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [50]:
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'mal')
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 1000 sentence pairs
Trimmed to 1000 sentence pairs
Counting words...
Counted words:
sanskrit 5841
mal 8586
['किन्तु यद्भक्ष्यं स्वर्गादागच्छत् तद् यदि कश्चिद् भुङ्क्त्ते तर्हि स न म्रियते।', 'തീവ്രമായി പരിശ്രമിക്കുന്ന യോഗിയാകട്ടെ പാപം നീങ്ങി അതിനുശേഷം അനേകജന്മങ്ങള്\u200d കൊണ്ടു സിദ്ധനായി പിന്നീട് പരമമായ ഗതിയെ പ്രാപിക്കുന്നു.']


In [51]:
print(input_lang.word2count['तथा'])

36


# **Seq2Seq Model**
---
A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

# **Encoder**

In [52]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Decoder**

In [53]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Training**


In [54]:
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(list(lang.word2index.values()))    
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **Training the model**

In [55]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [56]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [57]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    #training_pairs = training_pairs1
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

# **Evaluation**

In [58]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [59]:
def evaluate_test(encoder, decoder):
  test = []
  %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
  with open('test_sanskrit.sa', 'r', encoding="utf8") as f:
    test_text = [x.replace('\n', '') for x in f.readlines()]
  for sa_text in test_text:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    # print('<', output_sentence)
    # print('')
    test.append(output_sentence)
  with open('translated_malayalam.txt', 'w', encoding="utf8") as f:
    for line in test:
        f.write(line + '\n')

In [60]:
def translator(encoder, decoder):
    test_text = 'यत्र योगेश्वरः कृष्णो यत्र पार्थो धनुर्धरः। तत्र श्रीर्विजयो भूतिर्ध्रुवा नीतिर्मतिर्मम।।'

    output_words, attentions = evaluate(encoder, decoder, test_text)
    output_sentence = ' '.join(output_words)
    print(output_sentence)

In [61]:
 %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [62]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 75000, print_every=500)

0m 20s (- 50m 14s) (500 0%) 5.7711
0m 35s (- 43m 46s) (1000 1%) 6.4252
0m 50s (- 41m 22s) (1500 2%) 6.4132
1m 6s (- 40m 14s) (2000 2%) 6.4255
1m 21s (- 39m 29s) (2500 3%) 6.6301
1m 37s (- 39m 1s) (3000 4%) 6.6262
1m 53s (- 38m 31s) (3500 4%) 6.5802
2m 8s (- 38m 8s) (4000 5%) 6.3765
2m 24s (- 37m 51s) (4500 6%) 6.3270
2m 40s (- 37m 33s) (5000 6%) 6.2598
2m 56s (- 37m 15s) (5500 7%) 6.1708
3m 13s (- 36m 59s) (6000 8%) 6.0317
3m 28s (- 36m 41s) (6500 8%) 5.7309
3m 45s (- 36m 26s) (7000 9%) 5.6600
4m 1s (- 36m 11s) (7500 10%) 5.4229
4m 17s (- 35m 59s) (8000 10%) 5.3155
4m 34s (- 35m 44s) (8500 11%) 5.0961
4m 50s (- 35m 30s) (9000 12%) 4.8995
5m 7s (- 35m 17s) (9500 12%) 4.6272
5m 23s (- 35m 2s) (10000 13%) 4.5233
5m 39s (- 34m 47s) (10500 14%) 4.1620
5m 56s (- 34m 31s) (11000 14%) 3.9436
6m 12s (- 34m 16s) (11500 15%) 3.8464
6m 29s (- 34m 4s) (12000 16%) 3.5929
6m 46s (- 33m 50s) (12500 16%) 3.2955
7m 2s (- 33m 34s) (13000 17%) 2.9264
7m 19s (- 33m 20s) (13500 18%) 2.6939
7m 36s (- 33m 7s)

In [83]:
# saving the trained models
%cd C:\Users\Glitch\Sanskrit2mal
torch.save(encoder1.state_dict(), 'trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), 'trained_decoder.pth')

C:\Users\Glitch\Sanskrit2mal


In [84]:
evaluate_test(encoder1, attn_decoder1)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [85]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
output_text=[]
with open ('translated_malayalam.txt','r',encoding="utf8") as f:
  output_text.extend([f.readlines()])
print(output_text)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
[['ഹേ മനുഷ്യരിൽ ഈ അവന്റെ ഈ ഈ ഈ <EOS>\n', 'അപ്പോൾ ഒരു നല്ല നിങ്ങൾക്കു അയ്യോ കഷ്ടം! ഒരു <EOS>\n', 'രാജാവ് ഞാൻ ദുഃഖത്താൽ ഇവിടെ നിന്ന് നിങ്ങൾ ഇവിടെ നിന്ന് നിങ്ങൾ ഇവിടെ നിന്ന് നിങ്ങൾ എന്റെ <EOS>\n', 'ഹേ ഞാൻ എന്റെ എന്റെ ഈ എന്റെ ഈ <EOS>\n', 'ഹേ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഞാൻ ഈ ഞാൻ ഈ <EOS>\n', 'അവൻ ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ <EOS>\n', 'ഹേ രാമാ, ഈ ഈ ഈ ഈ ഈ ഈ <EOS>\n', 'അവൻ ഞാൻ ഈ ഞാൻ ഈ <EOS>\n', 'അവൻ ഞാൻ ഞാൻ ഈ ഞാൻ ഈ ഈ <EOS>\n', 'അവൻ എന്റെ അടുക്കൽ വന്നു ഞാൻ എന്തു എന്നു ഞാൻ അവൻ പറഞ്ഞു. <EOS>\n', 'അപ്പോൾ ഈ തന്റെ ഈ ഈ ഈ നീ എനിക്ക് എനിക്ക് കാണാൻ എനിക്ക് കാണാൻ കഴിഞ്ഞില്ല. <EOS>\n', 'ഒരു അധിപനായ കണ്ണുകളുള്ള രാമന്റെ ഈ കണ്ണുകളുള്ള ഒരു ജ്വലിക്കുന്ന രാമന്റെ <EOS>\n', 'ഹേ രാമാ, നിന്റെ ഈ ഈ ഈ ഈ ഈ <EOS>\n', 'രാമൻ വനവാസം രാമൻ തന്റെ വനവാസം വനവാസം ഈ വീണ്ടും വീണ്ടും വീണ്ടും വീണ്ടും വീണ്ടും ചെയ്തു. <EOS>\n', 'അവൻ ഞാൻ എന്റെ പിതാവിന്റെ ഈ ഒരു അവൻ അവൻ പറഞ്ഞു. <EOS>\n', 'യേശു എന്റെ എന്റെ ഞാൻ എന്റെ <EOS>\n', 'ഈ രാമാ, ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ ഈ <EOS>\n', 'ഓ, ശക്തനായ ശക്തന

In [86]:
translator(encoder1, attn_decoder1)

അവൻ ഞാൻ എല്ലാ ഈ തന്റെ കൊണ്ട് ഈ ചെയ്തു. <EOS>


In [58]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
import nltk
with open('test_malayalam.txt', 'r',  encoding="utf8") as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r',  encoding="utf8") as f: # translated Malayalam file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

bleu = nltk.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie )

print("bleu", bleu)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
bleu 0.0001488806327407329
